In [1]:
import os

from dotenv import load_dotenv
from simple_salesforce import Salesforce
import openai

import json
import logging
import copy

In [2]:
load_dotenv()

username = os.getenv('SALESFORCE_USERNAME')
password = os.getenv('SALESFORCE_PASSWORD')
security_token = os.getenv('SALESFORCE_SECURITY_TOKEN')

In [3]:
sf = Salesforce(username=username, password=password, security_token=security_token)

In [4]:
sf.search('FIND {BL Tech}')

OrderedDict([('searchRecords',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Lead'),
                                          ('url',
                                           '/services/data/v57.0/sobjects/Lead/00QHu00003crPvkMAE')])),
                            ('Id', '00QHu00003crPvkMAE')]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Account'),
                                          ('url',
                                           '/services/data/v57.0/sobjects/Account/001Hu00002xueELIAY')])),
                            ('Id', '001Hu00002xueELIAY')])])])

In [5]:
sf.query_all("SELECT Id, Name FROM Account")

{'records': [OrderedDict([('attributes',
                OrderedDict([('type', 'Account'),
                             ('url',
                              '/services/data/v57.0/sobjects/Account/001Hu00002xueEEIAY')])),
               ('Id', '001Hu00002xueEEIAY'),
               ('Name', 'JD Enterprises')]),
  OrderedDict([('attributes',
                OrderedDict([('type', 'Account'),
                             ('url',
                              '/services/data/v57.0/sobjects/Account/001Hu00002xueEFIAY')])),
               ('Id', '001Hu00002xueEFIAY'),
               ('Name', 'Smith Corp')]),
  OrderedDict([('attributes',
                OrderedDict([('type', 'Account'),
                             ('url',
                              '/services/data/v57.0/sobjects/Account/001Hu00002xueEGIAY')])),
               ('Id', '001Hu00002xueEGIAY'),
               ('Name', 'Em’s Store')]),
  OrderedDict([('attributes',
                OrderedDict([('type', 'Account'),
              

In [6]:
def execute_sosl(search: str):
    """Returns the result of a Salesforce search as a dict decoded from
    the Salesforce response JSON payload.
    Arguments:
    * search -- the fully formatted SOSL search string, e.g.
                `FIND {Waldo}`
    """
    from simple_salesforce import SalesforceError
    try:
        res = sf.search(search)
    except SalesforceError as err:
        return f"Error running SOSL query: {err}"
    return dict(res)


def execute_soql(query: str):
    """Returns the full set of results for the `query`. This is a
    convenience wrapper around `query(...)` and `query_more(...)`.
    The returned dict is the decoded JSON payload from the final call to
    Salesforce, but with the `totalSize` field representing the full
    number of results retrieved and the `records` list representing the
    full list of records retrieved.
    Arguments:
    * query -- the SOQL query to send to Salesforce, e.g.
               SELECT Id FROM Lead WHERE Email = "waldo@somewhere.com"
    """
    from simple_salesforce import SalesforceError
    try:
        res = sf.query_all(query)
    except SalesforceError as err:
        return f"Error running SOQL query: {err}"
    return res


In [7]:
logger = logging.getLogger(__name__)
openai.api_key = os.getenv("OPENAI_API_KEY")

def call_function(available_functions, response_message, messages):
    function_name = response_message["function_call"]["name"]
    arguments = response_message["function_call"]["arguments"]
    logger.info(
        f"LLM wants to call {function_name} function with arguments: {arguments}"
    )
    function_to_call = available_functions.get(function_name)
    if function_to_call is None:
        logger.error(
            f"LLM wanted to call function {function_name} that is not available"
        )
        messages.append(
            {
                "role": "user",
                "content": f"There is no function such: {function_name}. Available functions are: {','.join(available_functions.keys())}",
            }
        )
        return
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = function_to_call(**function_args)
    function_response = json.dumps(function_response)
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )
    logger.info(
        f"{function_name} for arguments {arguments} returned {function_response}"
    )


def chat(
    prompt, messages, available_functions, functions, max_turns=15, agent_name="agent"
):
    logger.info(f"Entering chat function for {agent_name} and prompt:\n {prompt}")
    messages_new = copy.deepcopy(messages)
    messages_new.append({"role": "user", "content": prompt})
    turns = 0
    while True:
        response = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            messages=messages_new,
            functions=functions,
            function_call="auto",
            temperature=0.00000001,
        )
        response_message = response["choices"][0]["message"]
        message_content = response["choices"][0]["message"]["content"]
        if message_content:
            logger.info(f"LLM responded with message: {message_content}\n")
        messages_new.append(response_message)
        if response_message.get("function_call"):
            call_function(available_functions, response_message, messages_new)
        else:
            logger.info(f"chat function for {agent_name} returned:\n {message_content}")
            break
        turns += 1
        if turns > max_turns:
            raise Exception("Reached max number of turns to LLM for single user query")
    messages[:] = messages_new
    return (
        response["choices"][0]["message"]["content"],
        response["usage"]["total_tokens"],
    )


In [9]:
class SalesforceAgent:
    SYSTEM_PROMPT = """
        You are SalesforceAgent, an AI specialized in managing Salesforce CRM data. Within Salesforce terminology, 
        'Accounts' refer to Companies and 'Opportunities' refer to Deals.
        Remember, a 'Closed Won' opportunity means the company has made a purchase, and an open opportunity is one that 
        is not 'Closed Won' or 'Closed Lost'.
        """

    PROMPT_TEMPLATE = """
        To address your request efficiently, please follow the streamlined process below:
        1. Understand the issue at hand and outline the essential steps and functions necessary for an accurate solution, aiming for minimalism in both.
        2. Implement the solution meticulously, focusing only on execution without the need to detail the plan to the user.
        
        Guidelines:
        - Refrain from creating non-existent functionalities.
        - Notify if the expected output cannot be generated.
        - When predicting forecast revenue, consider only open Opportunities, excluding those with a 'Closed Won' and 'Closed Lost' status.
        - If I ask about activities related to the deals/ opportunities - check tasks, events, calls.
        
        Problem: {}
        """

    def __init__(self):
        self.available_functions = {
            "execute_sosl": execute_sosl,
            "execute_soql": execute_soql,
        }
        self.functions = [
            {
                "name": "execute_sosl",
                "description": "Executes a SOSL search with the given search string.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "search": {
                            "type": "string",
                            "description": "The SOSL search string to be executed."
                        }
                    },
                    "required": ["search"]
                },
            },

            {
                "name": "execute_soql",
                "description": "Executes a SOQL query with the given query string.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The SOQL query string to be executed."
                        }
                    },
                    "required": ["query"]
                },
            }

        ]
        self.messages = [{"role": "system", "content": self.SYSTEM_PROMPT}]

    def chat(self, query):
        prompt = self.PROMPT_TEMPLATE.format(query)
        response, _ = chat(
            prompt,
            self.messages,
            self.available_functions,
            self.functions,
            agent_name="salesforce_agent",
        )
        return response


def ask_salesforce_agent(query):
    print("asking Salesforce Agent")
    return SalesforceAgent().chat(query)

In [10]:
if __name__ == "__main__":
    while True:
        query = input("Enter your query (or type 'exit' to end): ")
        if query.lower() == "exit":
            break
        response = ask_salesforce_agent(query)
        print("Agent:", response)

Enter your query (or type 'exit' to end):  list my deals


asking Salesforce Agent


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))